# TPF-02: Reprodução do Artigo DistilBERT

## Introdução

Este relatório documenta a reprodução dos experimentos do artigo "DistilBERT, a distilled version of BERT: smaller, faster, cheaper and lighter". O objetivo é replicar o *fine-tuning* do modelo DistilBERT na tarefa de análise de sentimentos **SST-2** Stanford Sentiment Treebank v2, que é uma tarefa de classificação binária positivo/negativo.

O objetivo é comparar a acurácia obtida nesse experimento com os resultados reportados no artigo original, utilizando as ferramentas do ecossistema Hugging Face.

## Configuração do Ambiente

O primeiro passo é garantir um ambiente de execução estável. O Google Colab, por padrão, pode conter versões de bibliotecas (`transformers`, `datasets`) que entram em conflito ou estão desatualizadas.

Para evitar os erros de `TypeError` relacionados a argumentos como `evaluation_strategy` ou falhas de cache, a célula abaixo executa uma instalação forçada `--force-reinstall` de versões específicas e compatíveis de todo o *stack* de bibliotecas.

Após a instalação, o *runtime* ambiente de execução é reiniciado `os.kill` para garantir que o notebook carregue as bibliotecas recém-instaladas na memória.

In [ ]:
# Força a instalação de versões específicas e compatíveis
!pip install -q --upgrade --force-reinstall numpy==1.26.4 pandas==2.2.2 pyarrow==15.0.2 \
datasets==2.20.0 transformers==4.44.2 evaluate==0.4.2 scikit-learn==1.5.1

# Força reiniciar o processo do Python
import os, sys, time
print("Reinstalação concluída. Reiniciando o runtime em 3s")
time.sleep(3)
os.kill(os.getpid(), 9)


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.7/57.7 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.5/40.5 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.1/75.1 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.7/12.7 MB 39.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 42.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 566.1/566.1 kB 36.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 308.4/308.4 kB 18.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.5/66.5 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 229.9/229.9 kB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 509.2/509.2 kB 23.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [6]:
# Desativa logs desnecessários
os.environ["WANDB_DISABLED"] = "true"
os.environ["WANDB_MODE"] = "offline"
os.environ["WANDB_SILENT"] = "true"
os.environ["TRANSFORMERS_NO_ADVISORY_WARNINGS"] = "true"

In [7]:
# Imports principais
import numpy as np
import evaluate
import datasets
import transformers

from datasets import load_dataset
from transformers import (
    AutoTokenizer,
    DataCollatorWithPadding,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer
)

print(f"Transformers: {transformers.__version__}")
print(f"Datasets: {datasets.__version__}")
print(f"Evaluate: {evaluate.__version__}")

Transformers: 4.44.2
Datasets: 2.20.0
Evaluate: 0.4.2


## Carregamento e Preparação de Dados

Após o reinício do ambiente, importamos as bibliotecas necessárias. Também desativamos os logs do `WandB` Weights & Biases para um *output* mais limpo.

Em seguida, foi executado o processo padrão de NLP:
1.  **Carregar Tokenizador:** Usamos o `AutoTokenizer` para carregar o `distilbert-base-uncased`, que é o tokenizador padrão do modelo.
2.  **Carregar Dataset:** Usamos `load_dataset` para baixar o `glue/sst2`.
3.  **Tokenizar:** Criamos uma função `tokenize_function` e a aplicamos ao dataset usando `.map()` para converter as sentenças de texto em IDs numéricos que o modelo entende.
4.  **Data Collator:** Instanciamos um `DataCollatorWithPadding` para agrupar dinamicamente as sentenças em lotes, aplicando *padding* (preenchimento) para que todas tenham o mesmo comprimento dentro de um lote.

In [8]:
# Escolha do modelo pré-treinado
checkpoint = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

print("\nCarregando dataset 'glue/sst2'")
raw_datasets = load_dataset("glue", "sst2")

# Função para tokenizar as sentenças
def tokenize_function(example):
    return tokenizer(example["sentence"], truncation=True)

print("Tokenizando os dados")
tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

/usr/local/lib/python3.12/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(



Carregando dataset 'glue/sst2'
Tokenizando os dados


Map:   0%|          | 0/872 [00:00<?, ? examples/s]

## Modelo, Métricas e Treinamento

Com os dados prontos, foram configurados os componentes do *fine-tuning*:

1.  **Métrica:** Carregada a métrica de `accuracy`, que é o padrão para a tarefa SST-2. A função `compute_metrics` formata as previsões do modelo logits para que a métrica possa ser calculada.
2.  **Modelo:** Carregada o `distilbert-base-uncased` usando `AutoModelForSequenceClassification`. É crucial definir `num_labels=2` para que o modelo saiba que deve classificar em duas categorias positiva e negativa.
3.  **Argumentos de Treino:** Definido os `TrainingArguments` hiperparâmetros. Segue os valores comuns do artigo, como `learning_rate=2e-5` e `num_train_epochs=3`.
    * `evaluation_strategy="epoch"`: Informa ao `Trainer` para rodar a avaliação no final de cada época.
    * `load_best_model_at_end=True`: Garante que, ao final do treino, o `Trainer` recarregue o *checkpoint* que teve a menor perda `eval_loss` no conjunto de validação.
4.  **Trainer:** Instanciado o `Trainer`, unindo o modelo, os dados, os argumentos e a função de métrica.

In [3]:
print("\n Carregando métrica")
metric = evaluate.load("accuracy")

def compute_metrics(eval_preds):
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

print("Carregando modelo")
model = AutoModelForSequenceClassification.from_pretrained(
    checkpoint,
    num_labels=2
)


 Carregando métrica


Carregando modelo


model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [9]:
print("\n Configurando parâmetros de treino")

training_args = TrainingArguments(
    output_dir="meu_modelo_distilbert_sst2",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,

    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    logging_steps=100,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

/usr/local/lib/python3.12/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).



 Configurando parâmetros de treino


## 5. Execução do Experimento e Resultados

Finalmente, iniciado o *fine-tuning* chamando `trainer.train()`. Após o término, chamamos `trainer.evaluate()` para obter os resultados finais do melhor modelo carregado automaticamente pelo `load_best_model_at_end=True`.

In [10]:
print("\n Iniciando o treinamento")
trainer.train()

print("\nAvaliando o modelo")
eval_results = trainer.evaluate()

print("\n Resultados Finais: ")
print(eval_results)


 Iniciando o treinamento


/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Epoch,Training Loss,Validation Loss,Accuracy
1,0.171100,0.310270,0.903670
2,0.122300,0.371548,0.904817


/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Epoch,Training Loss,Validation Loss,Accuracy
1,0.171100,0.310270,0.903670
2,0.122300,0.371548,0.904817
3,0.078700,0.424509,0.907110


/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)



Avaliando o modelo


/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)



 Resultados Finais: 
{'eval_loss': 0.3102695345878601, 'eval_accuracy': 0.9036697247706422, 'eval_runtime': 57.6134, 'eval_samples_per_second': 15.135, 'eval_steps_per_second': 0.955, 'epoch': 3.0}


## Resultados e Discussão

O treinamento foi executado por 3 épocas. Os logs de validação foram:

| Época | Training Loss | Validation Loss | Accuracy |
|:---:|:---:|:---:|:---:|
| 1 | 0.171100 | **0.310270** | 0.903670 |
| 2 | 0.122300 | 0.371548 | 0.904817 |
| 3 | 0.078700 | 0.424509 | 0.907110 |

### Análise

1.  **Acurácia Obtida:** A acurácia final foi de **90.37%**.
2.  **Comparação com o Artigo:** O artigo original do DistilBERT reporta uma acurácia de **91.3%** no conjunto de validação do SST-2.
3.  **Interpretação:** O resultado de 90.37% é **extremamente próximo** do resultado original 91.3%, com uma diferença de menos de 1%. Essa pequena variação é perfeitamente esperada em reproduções de *deep learning* e pode ser atribuída a fatores como:
    * Diferenças na inicialização aleatória dos pesos o "random seed".
    * Pequenas variações na implementação exata do otimizador ou do *scheduler* de *learning rate* entre as versões das bibliotecas.

### Ponto de Análise Overfitting

Um ponto interessante é o comportamento do `load_best_model_at_end=True`.

* A **Acurácia** continuou subindo até a Época 3 90.71%.
* Porém, a **Perda de Validação Validation Loss** atingiu seu ponto mínimo na **Época 1 0.310270** e começou a subir 0.371, 0.424, o que indica um leve *overfitting* nas épocas 2 e 3.
* Como o `Trainer` usa a `eval_loss` e não a acurácia como métrica padrão para salvar o melhor modelo, ele corretamente descartou os modelos das épocas 2 e 3 e **recarregou o modelo da Época 1**.
* É por isso que o resultado final `eval_accuracy: 0.9036...` é idêntico ao resultado da Época 1.